In [3]:
import os
import copy
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torchsummary import summary
from fastprogress import progress_bar
from .auto_encoder import encode_images, decode_images

### Load Dataset

In [ ]:
# define image transformation
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((128, 128)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# download dataset
train_dataset = torchvision.datasets.SUN397(root='dataset', transform=transform, download=True)

# # select classes
classes_idx = [idx for idx, label in enumerate(train_dataset.classes) if label in
    ['bedroom', 'beach', 'skyscraper', 'lighthouse', 'windmill', 'mountain', 'castle', 'rice_paddy', 'forest_path', 'bridge']]

# filter dataset by classes
train_dataset = torch.utils.data.Subset(train_dataset, [idx for idx, label in enumerate(train_dataset._labels) if label in classes_idx])

print('Dataset size:', len(train_dataset))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

# extract a batch of images
batch_images, batch_labels = next(iter(train_loader))
# display a grid of images
plt.figure(figsize=(16, 12))
plt.imshow(np.transpose(torchvision.utils.make_grid(batch_images[:8], padding=0).numpy(), (1, 2, 0)))
plt.axis('off')
plt.show()

In [ ]:
# replace labels
replace_dict = { label: idx for idx, label in enumerate(classes_idx) }

sample = torch.tensor([replace_dict.get(i, i) for i in batch_labels.tolist()])

print('Sample labels:', sample[:10])

#### Attention Blocks

In [ ]:
class SelfAttention(torch.nn.Module):
    def __init__(self, channels):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.mha = torch.nn.MultiheadAttention(channels, num_heads=1, batch_first=True)
        self.ln = torch.nn.LayerNorm([channels])
        self.ff_self = torch.nn.Sequential(
            torch.nn.LayerNorm([channels]),
            torch.nn.Linear(channels, channels),
            torch.nn.GELU(),
            torch.nn.Linear(channels, channels),
        )

    def forward(self, x):
        size = x.shape[-1]
        x = x.view(-1, self.channels, size * size).swapaxes(1, 2)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        attention_value = self.ff_self(attention_value) + attention_value
        return attention_value.swapaxes(2, 1).view(-1, self.channels, size, size)

#### Residual Blocks

In [ ]:
class DoubleConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        self.residual = residual
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            torch.nn.GroupNorm(1, mid_channels),
            torch.nn.GELU(),
            torch.nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            torch.nn.GroupNorm(1, out_channels)
        )

    def forward(self, x):
        if self.residual:
            return torch.nn.functional.gelu(x + self.double_conv(x))
        else:
            return self.double_conv(x)

In [ ]:
class Down(torch.nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        self.max_pool_conv = torch.nn.Sequential(
            torch.nn.MaxPool2d(2),
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels)
        )

        self.emb_layer = torch.nn.Sequential(
            torch.nn.SiLU(),
            torch.nn.Linear(emb_dim, out_channels)
        )

    def forward(self, x, t):
        x = self.max_pool_conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb

In [ ]:
class Up(torch.nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()

        self.up = torch.nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv = torch.nn.Sequential(
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels, in_channels // 2)
        )

        self.emb_layer = torch.nn.Sequential(
            torch.nn.SiLU(),
            torch.nn.Linear(emb_dim, out_channels)
        )

    def forward(self, x, skip_x, t):
        x = self.up(x)
        x = torch.cat([skip_x, x], dim=1)
        x = self.conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb

#### U-Net Transformer

In [ ]:
class UNetTransformer(torch.nn.Module):
    def __init__(self, c_in=3, c_out=3, time_dim=256, num_classes=None):
        super().__init__()
        # embeddings
        self.time_dim = time_dim
        self.label_emb = torch.nn.Embedding(num_classes, time_dim)
        # encoder
        self.inc = DoubleConv(c_in, 64)
        self.down1 = Down(64, 128)
        self.sa1 = SelfAttention(128)
        self.down2 = Down(128, 256)
        self.sa2 = SelfAttention(256)
        self.down3 = Down(256, 256)
        self.sa3 = SelfAttention(256)
        # bottleneck
        self.bot1 = DoubleConv(256, 512)
        self.bot2 = DoubleConv(512, 256)
        # decoder
        self.up1 = Up(512, 128)
        self.sa4 = SelfAttention(128)
        self.up2 = Up(256, 64)
        self.sa5 = SelfAttention(64)
        self.up3 = Up(128, 64)
        self.sa6 = SelfAttention(64)
        self.out_c = torch.nn.Conv2d(64, c_out, kernel_size=1)

    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (
            10000
            ** (torch.arange(0, channels, 2).float().cuda() / channels)
        )
        pos_enc_a = torch.sin(t.repeat(1, channels // 2) * inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels // 2) * inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc

    def forward(self, x, t, y=None):
        # time embedding
        t = t.unsqueeze(-1)
        t = self.pos_encoding(t, self.time_dim)
        # label embedding
        if y is not None:
            t += self.label_emb(y)
        # encoder
        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        x4 = self.sa3(x4)
        # bottleneck
        x4 = self.bot1(x4)
        x4 = self.bot2(x4)
        # decoder
        x = self.up1(x4, x3, t)
        x = self.sa4(x)
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        output = self.out_c(x)

        return output

In [ ]:
# model initializations
model = UNetTransformer(c_in=4, c_out=4, num_classes=10).float().cuda()
ema_model = copy.deepcopy(model).eval().requires_grad_(False)

sample = torch.rand((64, 4, 32, 32)).float().cuda()
output = model(sample)

print('Output shape:', output.shape)

In [ ]:
summary(model, input_size=(4, 32, 32))

#### Exponential Moving Average

In [ ]:
class EMA:
    def __init__(self, beta):
        super().__init__()
        self.beta = beta
        self.step = 0

    def update_model_average(self, ma_model, current_model):
        for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
            old_weight, up_weight = ma_params.data, current_params.data
            ma_params.data = self.update_average(old_weight, up_weight)

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new

    def step_ema(self, ema_model, model, step_start_ema=2000):
        if self.step < step_start_ema:
            self.reset_parameters(ema_model, model)
            self.step += 1
            return
        self.update_model_average(ema_model, model)
        self.step += 1

    def reset_parameters(self, ema_model, model):
        ema_model.load_state_dict(model.state_dict())

#### Train Diffusion Model

In [5]:
# hyperparameters
num_classes = 10
noise_steps = 1000
beta_start = 1e-4
beta_end = 0.02
img_size = 256
total_epochs = 100
learning_rate = 5e-3
log_every_epoch = 10

##### Noise Scheduler

In [6]:
beta = torch.linspace(beta_start, beta_end, noise_steps).float().cuda()
alpha = 1. - beta
alpha_hat = torch.cumprod(alpha, dim=0).float().cuda()

def noise_images(x, t):
    eps = torch.randn_like(x)
    sqrt_alpha_hat = torch.sqrt(alpha_hat[t])[:, None, None, None]
    sqrt_one_minus_alpha_hat = torch.sqrt(1 - alpha_hat[t])[:, None, None, None]

    return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * eps, eps

##### Optimizers

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=1e-5)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_loader), epochs=total_epochs)
ema = EMA(0.995)
scaler = torch.cuda.amp.GradScaler()

#### Callbacks

In [ ]:
def save_checkpoint(checkpoint_path="diffusion_models"):
    torch.save(model.state_dict(), os.path.join(checkpoint_path, f"ckpt.pt"))
    torch.save(ema_model.state_dict(), os.path.join(checkpoint_path, f"ema_ckpt.pt"))

def generate_samples(labels, cfg_scale=3):
    ema_model.eval()
    with torch.inference_mode():
        x = torch.randn((len(labels), 4, img_size, img_size)).float().cuda()
        for i in progress_bar(reversed(range(1, noise_steps)), total=noise_steps-1, leave=False):
            t = torch.full((len(labels),), i).long().cuda()
            predicted_noise = ema_model(x, t, labels)
            if cfg_scale > 0:
                unconditional_noise = ema_model(x, t, None)
                predicted_noise = torch.lerp(unconditional_noise, predicted_noise, cfg_scale)
            
            sample_alpha = alpha[t][:, None, None, None]
            sample_alpha_hat = alpha_hat[t][:, None, None, None]
            sample_beta = beta[t][:, None, None, None]
            sample_noise = torch.randn_like(x) if i > 1 else torch.zeros_like(x)

            x = 1 / torch.sqrt(sample_alpha) * (x - ((1 - sample_alpha) / (torch.sqrt(1 - sample_alpha_hat))) * predicted_noise) + torch.sqrt(sample_beta) * sample_noise
    # rescale generated image
    x = decode_images(x)
    x = (x.clamp(-1, 1) + 1) / 2
    x = (x * 255).type(torch.uint8)
    return x

def log_samples():
    labels = torch.arange(num_classes).long().cuda()
    sample_grid = generate_samples(labels)
    sample_grid = torchvision.utils.make_grid(sample_grid, nrow=(num_classes // 2))

    plt.imshow(sample_grid.cpu().permute(1, 2, 0).numpy(), vmin=0., vmax=1.)
    plt.axis('off')
    plt.show()

In [ ]:
for epoch in range(total_epochs):
    model.train()

    avg_loss = 0.
    p_bar = progress_bar(train_loader, leave=False)
    for images, labels in p_bar:
        images = encode_images(images)
        labels = torch.tensor([replace_dict.get(i, i) for i in labels.tolist()]).cuda()
        
        with torch.autocast("cuda") and torch.enable_grad():
            # noised images
            t = torch.randint(low=1, high=noise_steps, size=(images.shape[0],)).cuda()
            x_t, noise = noise_images(images, t)
            # dropout
            if np.random.random() < 0.1:
                labels = None
            # forward pass
            predicted_noise = model(x_t, t, labels)
            # calculate loss
            loss = torch.nn.functional.mse_loss(noise, predicted_noise)
        # backward pass
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        ema.step_ema(ema_model, model)
        scheduler.step()
        # logger
        p_bar.comment = f"loss: {loss.item():.2e}, learning_rate: {scheduler.get_last_lr()[0]:.2e}"
        avg_loss += loss
    print(f"loss: {avg_loss.mean().item():.2e}, learning_rate: {scheduler.get_last_lr()[0]:.2e}")

    # callbacks
    if (epoch + 1) % log_every_epoch == 0:
        log_samples()
        save_checkpoint(checkpoint_path="diffusion_models")

In [ ]:
save_checkpoint(checkpoint_path="diffusion_models")

#### Test Models

In [ ]:
log_samples()